- Yaici Walid
- Roberto Petoh Tsene

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

In [2]:
# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
import pandas as pd

data = pd.read_csv('/kaggle/input/layoutlm/medquad.csv',header=0, sep=',')
df = data.dropna(subset = ['question', 'answer'])

In [4]:
df_train=df.sample(frac=0.8,random_state=200)
df_test=df.drop(df_train.index)

In [5]:
questions = df_train['question']

answers = df_train['answer']

train_text = df_train['question'] + ' ' + df_train['answer']

In [15]:
train_file_path = '/kaggle/working/train_text.txt'
test_file_path = '/kaggle/working/test_question_answer.csv'
train_file_csv = '/kaggle/working/train_question_answer.csv'
def save_preprocessed_data(preprocessed_data, output_file):
    with open(output_file, "w") as f:
        f.write(preprocessed_data)
save_preprocessed_data("".join(train_text), '/kaggle/working/train_text.txt')
df_test.to_csv(test_file_path, index=True,header=True)
df_train.to_csv(train_file_csv, index=True,header=True)

In [7]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling,TextDataset

data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False, )

train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file_path,
        block_size=128)

training_args = TrainingArguments(
   output_dir="/kaggle/working/gpt2_healthcare_chatbot_v3",
   per_device_train_batch_size=32,
   num_train_epochs=2,
   save_strategy='no',
)
trainer = Trainer(
   model=model,
   args=training_args,
   data_collator=data_collator,
   train_dataset=train_dataset,
)

2024-02-11 15:27:01.078298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 15:27:01.078437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 15:27:01.196867: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling

In [8]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,2.076700
1000,1.845100
1500,1.733700
2000,1.696900


TrainOutput(global_step=2028, training_loss=1.835259117788582, metrics={'train_runtime': 1252.7587, 'train_samples_per_second': 51.78, 'train_steps_per_second': 1.619, 'total_flos': 4237372882944000.0, 'train_loss': 1.835259117788582, 'epoch': 2.0})

In [9]:
# Save the fine-tuned model
model.save_pretrained("/kaggle/working/gpt2_healthcare_chatbot_vf")
tokenizer.save_pretrained("/kaggle/working/gpt2_healthcare_chatbot_vf")

('/kaggle/working/gpt2_healthcare_chatbot_vf/tokenizer_config.json',
 '/kaggle/working/gpt2_healthcare_chatbot_vf/special_tokens_map.json',
 '/kaggle/working/gpt2_healthcare_chatbot_vf/vocab.json',
 '/kaggle/working/gpt2_healthcare_chatbot_vf/merges.txt',
 '/kaggle/working/gpt2_healthcare_chatbot_vf/added_tokens.json')

In [12]:
# Load the trained chatbot
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Replace 'path/to/your/model' with the actual path where you saved your fine-tuned model
model_path = '/kaggle/working/gpt2_healthcare_chatbot_vf'
# Load the fine-tuned GPT-2 model
model = GPT2LMHeadModel.from_pretrained(model_path)
model = model.to(device)
# Load the tokenizer associated with the model
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [13]:
# Input prompt for generation
prompt = "if i have headache what should i do ?"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_length=200, num_return_sequences=1, no_repeat_ngram_size=2, top_k=10, top_p=0.95, pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id, do_sample=True)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text

"if i have headache what should i do? If you have pain in the back of the neck or the side of your neck, you may need to take aspirin or other medicines to help control your pain.\n  \nIf your doctor doesnt prescribe painkillers or a pain reliever, it's important to talk to your child about taking medicines. If he or she has pain, ask him or her to stop.  He or She can also ask to be treated for pain that has come back. Treatments such as pain relievers, medicines, and other medications may help relieve pain and relieve symptoms.What are the genetic changes related to hereditary dyslipidemia? Mutations in many genes cause hereditary diabetes. The genes that cause diabetes are involved in cell development and development. They are called regulatory genes. Mutation in any of these genes alters the way a cell makes and stores sugar, which is important for normal cells to make insulin. When insulin is needed, cells use it to break down"